In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.misc
from scipy import ndimage
import matplotlib.pyplot as plt

### Parámetros de Mallado

In [3]:
### RECORREMOS UNA PROFUNDIDAD

dz = 8e-9  ## Paso en Profundidad  nm
start_z = 0
stop_z = 600e-9

### PARA LA SUPERFICIE X-Y
dy = 0.1e-6  ## Paso en Superficie um
start_y = -10e-6
stop_y = 10e-6

dx = 0.1e-6  ## Paso en Superficie um
start_x = -10e-6
stop_x = 10e-6

### EN UN TIEMPO 
dt = 5e-15  ## Time Step fs
start_t = 0
stop_t = 3e-12


x = np.linspace(start_x,stop_x,int(round((stop_x-start_x)/dx)))
y = np.linspace(start_y,stop_y,int(round((stop_y-start_y)/dy)))
z = np.linspace(start_z,stop_z,int(round((stop_z-start_z)/dz)))
t = np.linspace(start_t,stop_t,int(round((stop_t-start_t)/dt)))

X,Y,Z,tt= np.meshgrid(x,y,z,t)

In [ ]:
# ------------- Red Electrones -----------------
Ce_aux = 134.5 # J / (m^3 K^2)
ke = 235  # J /(m K s)
g = 5.69e17 # J /(m K^3 s)

#  ----------------------------------------------

# ------------- Red Lattice -----------------
Ci = 2.42e6  # J / (m^3 K)
#  -------------------------------------------


# ------------- Parámetros Láser -----------------
R = 0.88 # Reflectancia A.U.
delta = 20e-9 # Optical Penetration (nm)
ball_len = 100e-9 # Ballistic Length (nm)
F = 5*1e4 # Fluence J/cm^2 -> J/m^2 
tp = 100e-15 # FWHM fs
w0 = 5e-6 # Waist um
#  ------------------------------------------------------

# ------------- Parámetros Varios -----------------
ys = 3e-6 ## Perfil de Apertura del Láser (um)
T0 = 300 # K Temperatura Ambiente (K)
x0 = 0 # Posición central del Spot en x
y0 = 0 # Posición central del Spot en y
